<a href="https://colab.research.google.com/github/sampath11/DS-Unit-2-Linear-Models/blob/master/Sampath_Kumar_LS_DS_212_regression2_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 2*

---

# Regression 2

## Assignment

You'll continue to **predict how much it costs to rent an apartment in NYC,** using the dataset from renthop.com.

- [ ] Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
- [ ] Engineer at least two new features. (See below for explanation & ideas.)
- [ ] Fit a linear regression model with at least two features.
- [ ] Get the model's coefficients and intercept.
- [ ] Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.
- [ ] What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
- [ ] As always, commit your notebook to your fork of the GitHub repo.


#### [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)

> "Some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used." — Pedro Domingos, ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)

> "Coming up with features is difficult, time-consuming, requires expert knowledge. 'Applied machine learning' is basically feature engineering." — Andrew Ng, [Machine Learning and AI via Brain simulations](https://forum.stanford.edu/events/2011/2011slides/plenary/2011plenaryNg.pdf) 

> Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work. 

#### Feature Ideas
- Does the apartment have a description?
- How long is the description?
- How many total perks does each apartment have?
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths)
- Ratio of beds to baths
- What's the neighborhood, based on address or latitude & longitude?

## Stretch Goals
- [ ] If you want more math, skim [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression
- [ ] If you want more introduction, watch [Brandon Foltz, Statistics 101: Simple Linear Regression](https://www.youtube.com/watch?v=ZkjP5RJLQF4)
(20 minutes, over 1 million views)
- [ ] Add your own stretch goal(s) !

In [ ]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [ ]:
from datetime import date

In [ ]:
import numpy as np
import pandas as pd

# Read New York City apartment rental listing data
df = pd.read_csv(DATA_PATH+'apartments/renthop-nyc.csv')
assert df.shape == (49352, 34)

# Remove the most extreme 1% prices,
# the most extreme .1% latitudes, &
# the most extreme .1% longitudes
df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
        (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.95))]

In [ ]:
df.head()

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1.0,1,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1.0,4,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
df.tail()

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space
49347,1.0,2,2016-06-02 05:41:05,"30TH/3RD, MASSIVE CONV 2BR IN LUXURY FULL SERV...",E 30 St,40.7426,-73.9790,3200,230 E 30 St,medium,1,0,1,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
49348,1.0,1,2016-04-04 18:22:34,"HIGH END condo finishes, swimming pool, and ki...",Rector Pl,40.7102,-74.0163,3950,225 Rector Place,low,1,1,0,1,1,0,0,1,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1
49349,1.0,1,2016-04-16 02:13:40,Large Renovated One Bedroom Apartment with Sta...,West 45th Street,40.7601,-73.9900,2595,341 West 45th Street,low,1,1,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
49350,1.0,0,2016-04-08 02:13:33,Stylishly sleek studio apartment with unsurpas...,Wall Street,40.7066,-74.0101,3350,37 Wall Street,low,1,1,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
49351,1.0,2,2016-04-12 02:48:07,Look no further!!! This giant 2 bedroom apart...,Park Terrace East,40.8699,-73.9172,2200,30 Park Terrace East,low,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
df.shape

(48817, 34)

In [ ]:
# convert field to date_time

df['list_date'] = pd.to_datetime(df['created'], infer_datetime_format =True)
df['list_date'].head()

0   2016-06-24 07:54:24
1   2016-06-12 12:19:27
2   2016-04-17 03:26:41
3   2016-04-18 02:22:02
4   2016-04-28 01:32:41
Name: list_date, dtype: datetime64[ns]

In [ ]:
df['description'].head()

0    A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...
1                                                     
2    Top Top West Village location, beautiful Pre-w...
3    Building Amenities - Garage - Garden - fitness...
4    Beautifully renovated 3 bedroom flex 4 bedroom...
Name: description, dtype: object

In [ ]:

def keyword_cnt(x):
  keywords = ['Brand New', 'Renovated', 'Luxury', 'High End']
  cnt=0
  match=0
  if (type(x) != str):
    return(0)
  for num, word in enumerate (keywords):
     cnt = x.find(keywords[num])
     if (cnt != -1):
       ##print ("Got Match", x)
       match = match +1
     else:
      continue
  ##print ("Returning", match)    
  return(match)


In [ ]:

df['keyword_cnt'] = [keyword_cnt(x) for x in df['description']]


print (df['keyword_cnt'])

0        1
1        0
2        0
3        0
4        0
        ..
49347    1
49348    0
49349    1
49350    0
49351    0
Name: keyword_cnt, Length: 48817, dtype: int64


In [ ]:
date_filter = pd.Timestamp (date (2016, 5, 31))

df_train = df[df['list_date'] <= date_filter]

df_test = df[df['list_date'] > date_filter]

df_train['list_date'].head()

### setup dependent variable - y
y_var = 'price'

y_train = df_train[y_var]
y_test = df_test[y_var]

In [ ]:
## debug
print (y_train.head())
print (df_train.head())

2    2850
3    3275
4    3350
5    7995
6    3600
Name: price, dtype: int64
   bathrooms  bedrooms  ...           list_date keyword_cnt
2        1.0         1  ... 2016-04-17 03:26:41           0
3        1.0         1  ... 2016-04-18 02:22:02           0
4        1.0         4  ... 2016-04-28 01:32:41           0
5        2.0         4  ... 2016-04-19 04:24:47           0
6        1.0         2  ... 2016-04-27 03:19:56           0

[5 rows x 36 columns]


In [ ]:
df_test['list_date'].nunique

<bound method IndexOpsMixin.nunique of 0       2016-06-24 07:54:24
1       2016-06-12 12:19:27
11      2016-06-03 03:21:22
14      2016-06-01 03:11:01
24      2016-06-07 04:39:56
                ...        
49305   2016-06-16 04:20:46
49310   2016-06-21 06:25:35
49320   2016-06-02 13:24:18
49332   2016-06-06 01:22:44
49347   2016-06-02 05:41:05
Name: list_date, Length: 16978, dtype: datetime64[ns]>

We aready did the 1 categories in the prior exercise
1). Heuristics 


Lets continue with Baseline (Mean) and regression

In [ ]:
## Look at the baseline ( 0 features)

overall_mean = df['price'].mean()
print ("Overall mean is:", overall_mean)

train_mean = df_train['price'].mean()
test_mean = df_test['price'].mean()

print ("Train Mean is :", train_mean)
print ("Test mean is :", test_mean)

Overall mean is: 3579.5852469426636
Train Mean is : 3575.737743019567
Test mean is : 3586.8005065378725


In [ ]:
#1). Import the estimator class from scikit learn
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error , mean_squared_error

#2). Instantiate the class
model = LinearRegression()

#3. Arrange X features
features = ['bedrooms', 'bathrooms', 'dogs_allowed', 'dishwasher', 'laundry_in_unit', 'elevator', 'keyword_cnt']
print (f'Doing Linear Regression on these features:{features}')

X_train = df_train[features]
X_test = df_test[features]

#4. Fit the model

model.fit (X_train, y_train)

y_pred = model.predict (X_train)

print (X_train.head())
print (X_test.head())


Doing Linear Regression on these features:['bedrooms', 'bathrooms', 'dogs_allowed', 'dishwasher', 'laundry_in_unit', 'elevator', 'keyword_cnt']
   bedrooms  bathrooms  dogs_allowed  ...  laundry_in_unit  elevator  keyword_cnt
2         1        1.0             0  ...                0         0            0
3         1        1.0             0  ...                0         0            0
4         4        1.0             0  ...                0         0            0
5         4        2.0             0  ...                0         0            0
6         2        1.0             1  ...                0         0            0

[5 rows x 7 columns]
    bedrooms  bathrooms  dogs_allowed  ...  laundry_in_unit  elevator  keyword_cnt
0          3        1.5             0  ...                0         0            1
1          2        1.0             1  ...                0         1            0
11         1        1.0             0  ...                0         0            0
14        

In [ ]:
## Print coef returned
print ('Intercept returned :', model.intercept_)
coefficients = pd.Series ( model.coef_, features)

print (coefficients.to_string())

model.coef_ , model.intercept_

Intercept returned : 270.2714629246034
bedrooms            408.087111
bathrooms          1890.258548
dogs_allowed        156.928090
dishwasher           34.427546
laundry_in_unit     519.292254
elevator            450.002165
keyword_cnt         -49.914203


(array([ 408.08711143, 1890.25854806,  156.92808974,   34.42754559,
         519.2922538 ,  450.0021646 ,  -49.91420272]), 270.2714629246034)

In [ ]:
mae = mean_absolute_error (y_train,y_pred)
mse = mean_squared_error (y_train, y_pred)
rmse = np.sqrt(mse)


print ("Regression metrics for TRAIN data")
print (f'MAE - Mean Abs Error is: {mae:.2f}')
print (f"MSE - Mean Squared Error is: {mse:.2f}")
print (f"RMSE - Root Mean Squared Error is: {rmse:2f}")

# Get R2
train_r2 = model.score (X_train, y_train)
print (f"R^2 is:{train_r2:.2f}")

Regression metrics for TRAIN data
MAE - Mean Abs Error is: 778.90
MSE - Mean Squared Error is: 1409585.05
RMSE - Root Mean Squared Error is: 1187.259468
R^2 is:0.55


In [ ]:
## 5 Apply the model now to new data
y_pred = model.predict (X_test)

mae = mean_absolute_error (y_test, y_pred)
mse = mean_squared_error (y_test, y_pred)
rmse = np.sqrt(mse)

print ("REgression metrics for TEST data")
print (f'MAE - Mean Abs Error is: {mae:.2f}')
print (f"MSE - Mean Squared Err is: {mse:2f}")
print (f"RMSE - Root MSE is : {rmse:2f}")
# Get R^2
test_r2 = model.score (X_test, y_test)
print (f"R^2 is:{test_r2:.2f}")

REgression metrics for TEST data
MAE - Mean Abs Error is: 790.53
MSE - Mean Squared Err is: 1396870.278955
RMSE - Root MSE is : 1181.892668
R^2 is:0.55


##Function regression_3d below

In [ ]:
import itertools
import numpy as np
import plotly.express as px
import plotly.graph_objs as go
from sklearn.linear_model import LinearRegression

def regression_3d(df, x, y, z, num=100, **kwargs):
    """
    Visualize linear regression in 3D: 2 features + 1 target
    
    df : Pandas DataFrame
    x : string, feature 1 column in df
    y : string, feature 2 column in df
    z : string, target column in df
    num : integer, number of quantiles for each feature
    """
    
    # Plot data
    fig = px.scatter_3d(df, x, y, z, **kwargs)
    
    # Fit Linear Regression
    features = [x, y]
    target = z
    model = LinearRegression()
    model.fit(df[features], df[target])    
    
    # Define grid of coordinates in the feature space
    xmin, xmax = df[x].min(), df[x].max()
    ymin, ymax = df[y].min(), df[y].max()
    xcoords = np.linspace(xmin, xmax, num)
    ycoords = np.linspace(ymin, ymax, num)
    coords = list(itertools.product(xcoords, ycoords))
    
    # Make predictions for the grid
    predictions = model.predict(coords)
    Z = predictions.reshape(num, num).T
    
    # Plot predictions as a 3D surface (plane)
    fig.add_trace(go.Surface(x=xcoords, y=ycoords, z=Z))
    
    return fig

In [ ]:
regression_3d (df_train, x='bedrooms',  y='bathrooms',  
               z='price', 
               title = 'Prediction')

### The below are some test code for looping and finding string matches . -- IGNORE FOR ASSIGNMENT

In [ ]:
def f(x):
  kw = ['Geek', 'is', 'portal']
  cnt=0
  match=0
  print (cnt, kw[0])
  for num, word in enumerate (kw):
     cnt = x.find(kw[num])
     if (cnt != -1):
      print ("Got Match", x)
      match = match +1
  else:
   print (x)
   return(match)

In [ ]:
presidents = ["Washington", "Adams", "Jefferson", "Madison", "Monroe", "Adams", "Jackson"]
for num, name in enumerate(presidents, start=1):
    print("President {}: {}".format(num, name))

President 1: Washington
President 2: Adams
President 3: Jefferson
President 4: Madison
President 5: Monroe
President 6: Adams
President 7: Jackson


In [ ]:


# import pandas as pd
import pandas as pd
 
# list of strings
lst = {'lst':['Geeks is', 'For', 'Geeks', 'is', 
            'portal', 'for', 'Geeks']}
 
# Calling DataFrame constructor on list
df = pd.DataFrame(lst)
print(df)


print ("printing result")
result = [f(x) for x in df['lst']]
print (result)



        lst
0  Geeks is
1       For
2     Geeks
3        is
4    portal
5       for
6     Geeks
printing result
0 Geek
Got Match Geeks is
Got Match Geeks is
Geeks is
0 Geek
For
0 Geek
Got Match Geeks
Geeks
0 Geek
Got Match is
is
0 Geek
Got Match portal
portal
0 Geek
for
0 Geek
Got Match Geeks
Geeks
[2, 0, 1, 1, 1, 0, 1]
